In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("./data/iris.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [107]:
from sklearn.preprocessing import KBinsDiscretizer
X = data.iloc[:,:-1]
est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
est.fit(X)

Xt = est.transform(X)

Xt = Xt.astype(np.int)
data2 = data
data2.iloc[:,:-1] = Xt

In [108]:
print(np.unique(data["species"],return_counts=True))

(array(['setosa', 'versicolor', 'virginica'], dtype=object), array([50, 50, 50]))


In [109]:
np.unique(data["species"])

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [267]:
prob_dict = {}
np.unique(data["species"])

for species in np.unique(data["species"]): 
    prob_dict[species] = {}
    prob_dict[species]["feature"] = np.ones((3,data2.columns[:-1].shape[0])) # burası daha genel hale gelmeli 3 neye bağlı?
    prob_dict[species]["class_prob"] = 0


In [268]:
data2.columns[:-1]

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [270]:
prob_dict

{'setosa': {'feature': array([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
  'class_prob': 0},
 'versicolor': {'feature': array([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
  'class_prob': 0},
 'virginica': {'feature': array([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
  'class_prob': 0}}

In [276]:

for i in data2.values:
    name = i[-1]

    for j,e in enumerate(i[:-1]):

        prob_dict[name]["feature"][e,j]+=1
    prob_dict[name]["class_prob"] += 1


[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[1 2 0 0 'setosa']
1 0
2 1
0 2
0 3
[1 2 0 0 'setosa']
1 0
2 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[1 2 0 0 'setosa']
1 0
2 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 2 0 0 'setosa']
0 0
2 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0 0
1 1
0 2
0 3
[0 1 0 0 'setosa']
0

In [277]:
prob_dict

{'setosa': {'feature': array([[46.,  2., 51., 51.],
         [ 6., 35.,  1.,  1.],
         [ 1., 16.,  1.,  1.]]),
  'class_prob': 50},
 'versicolor': {'feature': array([[ 7., 22.,  1.,  1.],
         [39., 30., 49., 49.],
         [ 7.,  1.,  3.,  3.]]),
  'class_prob': 50},
 'virginica': {'feature': array([[ 2., 12.,  1.,  1.],
         [28., 37.,  7.,  5.],
         [23.,  4., 45., 47.]]),
  'class_prob': 50}}

In [278]:
a = np.zeros((3,4))
a[2,0] = 5
print(a)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [5. 0. 0. 0.]]


In [279]:
 data2.shape[0]

150

In [280]:
for i in prob_dict.keys():
    print(i)
    prob_dict[i]["feature"]/= (prob_dict[i]["class_prob"]+1) # en baştaki +1 den burada da standartize ederken 1 eklkiyoruz
    prob_dict[i]["class_prob"]/= data2.shape[0]

setosa
versicolor
virginica


In [281]:
prob_dict

{'setosa': {'feature': array([[0.90196078, 0.03921569, 1.        , 1.        ],
         [0.11764706, 0.68627451, 0.01960784, 0.01960784],
         [0.01960784, 0.31372549, 0.01960784, 0.01960784]]),
  'class_prob': 0.3333333333333333},
 'versicolor': {'feature': array([[0.1372549 , 0.43137255, 0.01960784, 0.01960784],
         [0.76470588, 0.58823529, 0.96078431, 0.96078431],
         [0.1372549 , 0.01960784, 0.05882353, 0.05882353]]),
  'class_prob': 0.3333333333333333},
 'virginica': {'feature': array([[0.03921569, 0.23529412, 0.01960784, 0.01960784],
         [0.54901961, 0.7254902 , 0.1372549 , 0.09803922],
         [0.45098039, 0.07843137, 0.88235294, 0.92156863]]),
  'class_prob': 0.3333333333333333}}

In [306]:
prob_matrix = np.zeros((data2.shape[0],3))

for k,i in enumerate(data2.iloc):
    spec = i["species"]
    for j in range(len(np.unique(data["species"]))):
        name = np.unique(data["species"])[j]
        prob = prob_dict[name]["class_prob"] * prob_dict[name]["feature"][i.values[:-1].astype(np.int),np.arange(4)].prod()
        prob_matrix[k,j] = prob
    

In [309]:
prob_matrix

array([[2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [9.43226964e-02, 3.44902292e-07, 3.94174048e-07],
       [9.43226964e-02, 3.44902292e-07, 3.94174048e-07],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [9.43226964e-02, 3.44902292e-07, 3.94174048e-07],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [1.23029604e-02, 1.92159849e-06, 5.51843668e-06],
       [1.23029604e-02, 1.92159849e-06, 5.51843668e-06],
       [9.43226964e-02, 3.44902292e-07, 3.94174048e-07],
       [2.06330898e-01, 1.03470

In [313]:
arg_max = np.argmax(prob_matrix,axis=1)

In [319]:
arg_max

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [315]:
b = np.zeros(150)
b[51:100]=1
b[100:] = 2

In [318]:
(b == arg_max).sum()

143

In [320]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()

In [323]:
clf.fit(Xt, data2.iloc[:,-1].values)

CategoricalNB()

In [325]:
clf.predict_proba(Xt)

array([[9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99992164e-01, 3.65659171e-06, 4.17896196e-06],
       [9.99992164e-01, 3.65659171e-06, 4.17896196e-06],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99992164e-01, 3.65659171e-06, 4.17896196e-06],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99395630e-01, 1.56095530e-04, 4.48274344e-04],
       [9.99395630e-01, 1.56095530e-04, 4.48274344e-04],
       [9.99992164e-01, 3.65659171e-06, 4.17896196e-06],
       [9.99932185e-01, 5.01445